Com base na EDA que fiz, preciso de repetir todo o procedimento quando recebo novos dados

In [84]:
import pandas as pd
import numpy as np


# Lista final de features que o modelo treinado espera, NA ORDEM CORRETA.
# Esta lista foi definida no final da sua EDA.
FEATURES_FINAIS_PARA_MODELO = [
    'total_issues_created_log',
    'issue_participation_log',
    'issue_resolution_rate_created_log',
    'total_review_activity_log',
    'merged_ratio_log',
    'sum_lines_added_log',
    'interval',
    'branches_created',
]

# Epsilon para evitar divisão por zero nos ratios
EPSILON = 1e-6

# Colunas a serem removidas inicialmente se existirem
COLS_TO_DROP_INITIAL = [
    "project_id",
    "group_id",
    "mention_handle",
    "last_minute_commits", # Foi identificada como sempre zero
    "sum_lines_per_commit" # Foi removida devido à alta correlação/redundância
]

In [85]:
# ==============================================================================
# FUNÇÃO DE PRÉ-PROCESSAMENTO
# ==============================================================================

def preprocess_data_for_prediction(df_new_raw):
    df_processed = df_new_raw.copy()
    print(f"Pré-processamento: Dimensões iniciais dos novos dados: {df_processed.shape}")

    # --- 1. Remover colunas irrelevantes ---
    print("Pré-processamento: Removendo colunas irrelevantes...")
    for col in COLS_TO_DROP_INITIAL:
        if col in df_processed.columns:
            df_processed.drop(columns=[col], inplace=True)
            print(f"  Coluna '{col}' removida.")
    print(f"Pré-processamento: Dimensões após remoção inicial: {df_processed.shape}")


    # --- 2. Criar features _log diretamente necessárias para o modelo ---
    # (Estas são features que entram no modelo como _log e não são derivadas de ratios complexos)
    print("Pré-processamento: Criando features _log diretas...")

    # sum_lines_added_log
    if 'sum_lines_added' in df_processed.columns:
        df_processed['sum_lines_added_log'] = np.log1p(df_processed['sum_lines_added'])
        print("  Feature 'sum_lines_added_log' criada.")
    else:
        print("  AVISO: Coluna 'sum_lines_added' não encontrada. 'sum_lines_added_log' não será criada diretamente aqui (será preenchida no final se faltar).")

    # total_issues_created_log
    if 'total_issues_created' in df_processed.columns:
        df_processed['total_issues_created_log'] = np.log1p(df_processed['total_issues_created'])
        print("  Feature 'total_issues_created_log' criada.")
    else:
        print("  AVISO: Coluna 'total_issues_created' não encontrada. 'total_issues_created_log' não será criada diretamente aqui.")

    # issue_participation_log
    if 'issue_participation' in df_processed.columns:
        df_processed['issue_participation_log'] = np.log1p(df_processed['issue_participation'])
        print("  Feature 'issue_participation_log' criada.")
    else:
        print("  AVISO: Coluna 'issue_participation' não encontrada. 'issue_participation_log' não será criada diretamente aqui.")


    # --- 3. Engenharia de Features (Ratios e Combinadas com Log) ---
    print("Pré-processamento: Iniciando engenharia de features (ratios, combinadas)...")

    # Merged Ratio Log
    if 'merged_requests' in df_processed.columns and 'total_merge_requests' in df_processed.columns:
        merged_ratio_raw = df_processed['merged_requests'] / (df_processed['total_merge_requests'] + EPSILON)
        df_processed['merged_ratio_log'] = np.log1p(merged_ratio_raw)
        print("  Feature 'merged_ratio_log' criada.")
    else:
        print("  AVISO: Colunas 'merged_requests' ou 'total_merge_requests' não encontradas. 'merged_ratio_log' não será criada (será preenchida no final se faltar).")
        df_processed['merged_ratio_log'] = np.log1p(0) # Preenchimento padrão se faltar

    # Issue Resolution Rate Created Log
    if 'issues_resolved' in df_processed.columns and 'total_issues_created' in df_processed.columns:
        issue_res_rate_created_raw = df_processed['issues_resolved'] / (df_processed['total_issues_created'] + EPSILON)
        df_processed['issue_resolution_rate_created_log'] = np.log1p(issue_res_rate_created_raw)
        print("  Feature 'issue_resolution_rate_created_log' criada.")
    else:
        print("  AVISO: Colunas 'issues_resolved' ou 'total_issues_created' não encontradas. 'issue_resolution_rate_created_log' não será criada (será preenchida no final se faltar).")
        df_processed['issue_resolution_rate_created_log'] = np.log1p(0) # Preenchimento padrão

    # Total Review Activity Log
    if 'review_comments_given' in df_processed.columns and 'review_comments_received' in df_processed.columns:
        total_review_raw = df_processed['review_comments_given'] + df_processed['review_comments_received']
        df_processed['total_review_activity_log'] = np.log1p(total_review_raw)
        print("  Feature 'total_review_activity_log' criada.")
    else:
        print("  AVISO: Colunas 'review_comments_given' ou 'review_comments_received' não encontradas. 'total_review_activity_log' não será criada (será preenchida no final se faltar).")
        df_processed['total_review_activity_log'] = np.log1p(0) # Preenchimento padrão

    # --- 4. Garantir que todas as features finais estão presentes e selecionar ---
    #    As features 'interval' e 'branches_created' são usadas como estão (se existirem).
    print("Pré-processamento: Selecionando e ordenando features finais...")
    missing_features_in_final_step = []
    for final_feature in FEATURES_FINAIS_PARA_MODELO:
        if final_feature not in df_processed.columns:
            print(f"  AVISO FINAL: Feature final '{final_feature}' não encontrada nos dados processados. Preenchendo com 0.")
            df_processed[final_feature] = 0 # Ou outra estratégia de imputação (ex: média do treino)
            missing_features_in_final_step.append(final_feature)
    
    if missing_features_in_final_step:
        print(f"  Resumo de features finais que foram preenchidas com 0: {missing_features_in_final_step}")

    # Selecionar apenas as colunas finais na ordem correta
    # Adiciona colunas faltantes com 0 se, mesmo após tudo, elas não existirem (caso extremo)
    for col in FEATURES_FINAIS_PARA_MODELO:
        if col not in df_processed:
            df_processed[col] = 0

    df_model_ready = df_processed[FEATURES_FINAIS_PARA_MODELO]
    print(f"Pré-processamento: Dimensões finais dos dados prontos para o modelo (antes do scaler): {df_model_ready.shape}")
    
    return df_model_ready


---

In [86]:
#import pandas as pd
#import glob

#all_files = sorted(glob.glob("student_features_interval_*.csv"))
#df_all = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
## guardar um csv com estes dados todos agrupados
#df_all.to_csv("student_features_interval_all.csv", index=False)

In [87]:
import pandas as pd

df = pd.read_csv("student_features_interval_all.csv")  # Substitua pelo nome do seu arquivo

In [88]:
X = df.drop(columns=["Final Grade"])  # Features
y = df["Final Grade"]  # Target

In [89]:
print("y.value_counts(normalize=True):\n")
print(y.value_counts(normalize=True))


y.value_counts(normalize=True):

Final Grade
15    0.232126
12    0.183844
16    0.128134
10    0.098422
17    0.087279
13    0.077066
14    0.058496
11    0.038997
18    0.027855
7     0.022284
19    0.013928
1     0.008357
0     0.007428
2     0.004643
4     0.004643
9     0.003714
8     0.002786
Name: proportion, dtype: float64


In [90]:
from sklearn.model_selection import StratifiedKFold

n_splits = 10 
cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

Existe um claro desblanceamento!!! 
- A classe mais frequente (nota 15) representa 23.2% dos dados, enquanto classes como 8, 9, 2 e 4 representam menos de 0.5% cada.
- Há uma grande variação na frequência das classes, o que indica um forte desbalanceamento.

É importante normalizarmos os dados, principalmente porque temos features numéricas com escalas muito diferentes.

# Modelo

### abordagem 1

In [91]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=42)

In [92]:
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
#from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from imblearn.pipeline import Pipeline

smote = SMOTE(random_state=42, k_neighbors=1)

# Criar um pipeline que inclui o pré-processamento, escalonamento e o modelo
pipeline = Pipeline([
    ('preprocessing', FunctionTransformer(preprocess_data_for_prediction, validate=False)),  # Pré-processamento
    ('scaling', StandardScaler()),  # Escalonamento
    ('smote', smote),  # SMOTE para oversampling
    ('model', model)  # Modelo
])

In [93]:
f1_scores, mae_scores, mse_scores, classification_reports = [], [], [], []

In [ ]:
from sklearn.metrics import classification_report, f1_score, mean_absolute_error, mean_squared_error

for train_index, test_index in cv.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)

    # Calcular e armazenar as métricas
    f1 = f1_score(y_test, y_pred, average='macro')
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)

    f1_scores.append(f1)
    mae_scores.append(mae)
    mse_scores.append(mse)

    report = classification_report(y_test, y_pred)
    classification_reports.append(report)



In [95]:
# Imprimir as métricas médias
print("F1-Score (Macro) por fold:", f1_scores)
print(f"F1-Score (Macro) Médio: {np.mean(f1_scores):.4f}")
print(f"F1-Score (Macro) Desvio Padrão: {np.std(f1_scores):.4f}")

print("MAE por fold:", mae_scores)
print(f"MAE Médio: {np.mean(mae_scores):.4f}")
print(f"MAE Desvio Padrão: {np.std(mae_scores):.4f}")

print("MSE por fold:", mse_scores)
print(f"MSE Médio: {np.mean(mse_scores):.4f}")
print(f"MSE Desvio Padrão: {np.std(mse_scores):.4f}")

F1-Score (Macro) por fold: [0.4878060017935569, 0.5482832847997217, 0.5522682178932179, 0.5215433248771206, 0.4944847020933978, 0.5836566332218507, 0.4011335770817732, 0.43987234435019174, 0.3812196208421117, 0.5245130753503991]
F1-Score (Macro) Médio: 0.4935
F1-Score (Macro) Desvio Padrão: 0.0635
MAE por fold: [1.2592592592592593, 1.4166666666666667, 1.2037037037037037, 1.462962962962963, 1.2962962962962963, 1.0833333333333333, 1.6203703703703705, 1.5046728971962617, 1.6822429906542056, 1.6355140186915889]
MAE Médio: 1.4165
MAE Desvio Padrão: 0.1912
MSE por fold: [4.203703703703703, 8.50925925925926, 5.648148148148148, 8.518518518518519, 6.055555555555555, 3.7685185185185186, 6.87962962962963, 6.140186915887851, 11.813084112149532, 6.850467289719626]
MSE Médio: 6.8387
MSE Desvio Padrão: 2.2192


In [96]:
# Imprimir o relatório de classificação consolidado
print("\nRelatório de Classificação Consolidado:")
print("------------------------------------")
for i, report in enumerate(classification_reports):
    print(f"Fold {i+1}:\n{report}")
    print("------------------------------------")


Relatório de Classificação Consolidado:
------------------------------------
Fold 1:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       1.00      1.00      1.00         1
           7       0.00      0.00      0.00         2
           9       0.00      0.00      0.00         1
          10       0.50      0.45      0.48        11
          11       1.00      0.50      0.67         4
          12       0.54      0.65      0.59        20
          13       0.27      0.38      0.32         8
          14       0.31      0.57      0.40         7
          15       0.42      0.32      0.36        25
          16       0.47      0.50      0.48        14
          17       0.67      0.44      0.53         9
          18       1.00      1.00      1.00         3
          19       0.00      0.00      0.00         2

    accuracy                           0.47       108
   macro avg       0.51      0.49      0.49     

### abordagem 2 - RandomOverSampler (Sem Vizinhos Próximos):

In [97]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)
pipeline = Pipeline([
    ('preprocessing', FunctionTransformer(preprocess_data_for_prediction, validate=False)),
    ('scaling', StandardScaler()),
    ('ros', ros),  # Substitui SMOTE por RandomOverSampler
    ('model', model)
])

In [ ]:
for train_index, test_index in cv.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)

    # Calcular e armazenar as métricas
    f1 = f1_score(y_test, y_pred, average='macro')
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)

    f1_scores.append(f1)
    mae_scores.append(mae)
    mse_scores.append(mse)

    report = classification_report(y_test, y_pred)
    classification_reports.append(report)

In [99]:
# Imprimir as métricas médias
print("F1-Score (Macro) por fold:", f1_scores)
print(f"F1-Score (Macro) Médio: {np.mean(f1_scores):.4f}")
print(f"F1-Score (Macro) Desvio Padrão: {np.std(f1_scores):.4f}")

print("MAE por fold:", mae_scores)
print(f"MAE Médio: {np.mean(mae_scores):.4f}")
print(f"MAE Desvio Padrão: {np.std(mae_scores):.4f}")

print("MSE por fold:", mse_scores)
print(f"MSE Médio: {np.mean(mse_scores):.4f}")
print(f"MSE Desvio Padrão: {np.std(mse_scores):.4f}")

F1-Score (Macro) por fold: [0.4878060017935569, 0.5482832847997217, 0.5522682178932179, 0.5215433248771206, 0.4944847020933978, 0.5836566332218507, 0.4011335770817732, 0.43987234435019174, 0.3812196208421117, 0.5245130753503991, 0.48298665620094194, 0.5300436834310203, 0.5367387674133198, 0.42535583641060315, 0.47610159212132896, 0.5409639126305793, 0.30233790485891326, 0.4580303153007234, 0.39562779523610336, 0.5359062345301938]
F1-Score (Macro) Médio: 0.4809
F1-Score (Macro) Desvio Padrão: 0.0696
MAE por fold: [1.2592592592592593, 1.4166666666666667, 1.2037037037037037, 1.462962962962963, 1.2962962962962963, 1.0833333333333333, 1.6203703703703705, 1.5046728971962617, 1.6822429906542056, 1.6355140186915889, 1.2592592592592593, 1.2407407407407407, 1.3981481481481481, 1.5648148148148149, 1.5092592592592593, 1.3981481481481481, 1.7777777777777777, 1.4485981308411215, 1.6261682242990654, 1.588785046728972]
MAE Médio: 1.4488
MAE Desvio Padrão: 0.1784
MSE por fold: [4.203703703703703, 8.509

In [100]:
# Imprimir o relatório de classificação consolidado
print("\nRelatório de Classificação Consolidado:")
print("------------------------------------")
for i, report in enumerate(classification_reports):
    print(f"Fold {i+1}:\n{report}")
    print("------------------------------------")


Relatório de Classificação Consolidado:
------------------------------------
Fold 1:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       1.00      1.00      1.00         1
           7       0.00      0.00      0.00         2
           9       0.00      0.00      0.00         1
          10       0.50      0.45      0.48        11
          11       1.00      0.50      0.67         4
          12       0.54      0.65      0.59        20
          13       0.27      0.38      0.32         8
          14       0.31      0.57      0.40         7
          15       0.42      0.32      0.36        25
          16       0.47      0.50      0.48        14
          17       0.67      0.44      0.53         9
          18       1.00      1.00      1.00         3
          19       0.00      0.00      0.00         2

    accuracy                           0.47       108
   macro avg       0.51      0.49      0.49     

### abordagem 3 -  Sem Reamostragem (Linha de Base Sem Correção de Desbalanceamento)

In [101]:
pipeline = Pipeline([
    ('preprocessing', FunctionTransformer(preprocess_data_for_prediction, validate=False)),
    ('scaling', StandardScaler()),
    ('model', model)  # Sem reamostragem
])

In [ ]:
for train_index, test_index in cv.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)

    # Calcular e armazenar as métricas
    f1 = f1_score(y_test, y_pred, average='macro')
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)

    f1_scores.append(f1)
    mae_scores.append(mae)
    mse_scores.append(mse)

    report = classification_report(y_test, y_pred)
    classification_reports.append(report)

In [103]:
# Imprimir as métricas médias
print("F1-Score (Macro) por fold:", f1_scores)
print(f"F1-Score (Macro) Médio: {np.mean(f1_scores):.4f}")
print(f"F1-Score (Macro) Desvio Padrão: {np.std(f1_scores):.4f}")

print("MAE por fold:", mae_scores)
print(f"MAE Médio: {np.mean(mae_scores):.4f}")
print(f"MAE Desvio Padrão: {np.std(mae_scores):.4f}")

print("MSE por fold:", mse_scores)
print(f"MSE Médio: {np.mean(mse_scores):.4f}")
print(f"MSE Desvio Padrão: {np.std(mse_scores):.4f}")

F1-Score (Macro) por fold: [0.4878060017935569, 0.5482832847997217, 0.5522682178932179, 0.5215433248771206, 0.4944847020933978, 0.5836566332218507, 0.4011335770817732, 0.43987234435019174, 0.3812196208421117, 0.5245130753503991, 0.48298665620094194, 0.5300436834310203, 0.5367387674133198, 0.42535583641060315, 0.47610159212132896, 0.5409639126305793, 0.30233790485891326, 0.4580303153007234, 0.39562779523610336, 0.5359062345301938, 0.4958239436125327, 0.5752231027650959, 0.5247387226192801, 0.470095040492654, 0.46116544046231545, 0.5484842946478168, 0.35187590187590184, 0.4666895196306961, 0.4454648037054508, 0.5336397477651348]
F1-Score (Macro) Médio: 0.4831
F1-Score (Macro) Desvio Padrão: 0.0668
MAE por fold: [1.2592592592592593, 1.4166666666666667, 1.2037037037037037, 1.462962962962963, 1.2962962962962963, 1.0833333333333333, 1.6203703703703705, 1.5046728971962617, 1.6822429906542056, 1.6355140186915889, 1.2592592592592593, 1.2407407407407407, 1.3981481481481481, 1.5648148148148149, 1

In [104]:
# Imprimir o relatório de classificação consolidado
print("\nRelatório de Classificação Consolidado:")
print("------------------------------------")
for i, report in enumerate(classification_reports):
    print(f"Fold {i+1}:\n{report}")
    print("------------------------------------")


Relatório de Classificação Consolidado:
------------------------------------
Fold 1:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       1.00      1.00      1.00         1
           7       0.00      0.00      0.00         2
           9       0.00      0.00      0.00         1
          10       0.50      0.45      0.48        11
          11       1.00      0.50      0.67         4
          12       0.54      0.65      0.59        20
          13       0.27      0.38      0.32         8
          14       0.31      0.57      0.40         7
          15       0.42      0.32      0.36        25
          16       0.47      0.50      0.48        14
          17       0.67      0.44      0.53         9
          18       1.00      1.00      1.00         3
          19       0.00      0.00      0.00         2

    accuracy                           0.47       108
   macro avg       0.51      0.49      0.49     

### abordgaem 4 -  Ajustar os Pesos das Classes (class_weight='balanced')

In [105]:
model = RandomForestClassifier(random_state=42, class_weight='balanced')
pipeline = Pipeline([
    ('preprocessing', FunctionTransformer(preprocess_data_for_prediction, validate=False)),
    ('scaling', StandardScaler()),
    ('model', model)
])

In [ ]:
for train_index, test_index in cv.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)

    # Calcular e armazenar as métricas
    f1 = f1_score(y_test, y_pred, average='macro')
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)

    f1_scores.append(f1)
    mae_scores.append(mae)
    mse_scores.append(mse)

    report = classification_report(y_test, y_pred)
    classification_reports.append(report)

In [107]:
# Imprimir as métricas médias
print("F1-Score (Macro) por fold:", f1_scores)
print(f"F1-Score (Macro) Médio: {np.mean(f1_scores):.4f}")
print(f"F1-Score (Macro) Desvio Padrão: {np.std(f1_scores):.4f}")

print("MAE por fold:", mae_scores)
print(f"MAE Médio: {np.mean(mae_scores):.4f}")
print(f"MAE Desvio Padrão: {np.std(mae_scores):.4f}")

print("MSE por fold:", mse_scores)
print(f"MSE Médio: {np.mean(mse_scores):.4f}")
print(f"MSE Desvio Padrão: {np.std(mse_scores):.4f}")

F1-Score (Macro) por fold: [0.4878060017935569, 0.5482832847997217, 0.5522682178932179, 0.5215433248771206, 0.4944847020933978, 0.5836566332218507, 0.4011335770817732, 0.43987234435019174, 0.3812196208421117, 0.5245130753503991, 0.48298665620094194, 0.5300436834310203, 0.5367387674133198, 0.42535583641060315, 0.47610159212132896, 0.5409639126305793, 0.30233790485891326, 0.4580303153007234, 0.39562779523610336, 0.5359062345301938, 0.4958239436125327, 0.5752231027650959, 0.5247387226192801, 0.470095040492654, 0.46116544046231545, 0.5484842946478168, 0.35187590187590184, 0.4666895196306961, 0.4454648037054508, 0.5336397477651348, 0.4805891184838553, 0.5095636922230374, 0.5380373440026417, 0.4983272035744618, 0.4933003364389234, 0.5494109721695929, 0.3292491364461762, 0.44435681555687434, 0.39251501195569066, 0.48728377710021437]
F1-Score (Macro) Médio: 0.4804
F1-Score (Macro) Desvio Padrão: 0.0662
MAE por fold: [1.2592592592592593, 1.4166666666666667, 1.2037037037037037, 1.462962962962963

In [108]:
# Imprimir o relatório de classificação consolidado
print("\nRelatório de Classificação Consolidado:")
print("------------------------------------")
for i, report in enumerate(classification_reports):
    print(f"Fold {i+1}:\n{report}")
    print("------------------------------------")


Relatório de Classificação Consolidado:
------------------------------------
Fold 1:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       1.00      1.00      1.00         1
           7       0.00      0.00      0.00         2
           9       0.00      0.00      0.00         1
          10       0.50      0.45      0.48        11
          11       1.00      0.50      0.67         4
          12       0.54      0.65      0.59        20
          13       0.27      0.38      0.32         8
          14       0.31      0.57      0.40         7
          15       0.42      0.32      0.36        25
          16       0.47      0.50      0.48        14
          17       0.67      0.44      0.53         9
          18       1.00      1.00      1.00         3
          19       0.00      0.00      0.00         2

    accuracy                           0.47       108
   macro avg       0.51      0.49      0.49     

---

Com base nos resultados atuais, a melhor abordagem até agora é SMOTE (k_neighbors=1)

In [120]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'model__n_estimators': [100, 200, 300],
    'model__max_depth': [5, 10, 15],
    'model__min_samples_split': [2, 5, 10],
    'model__min_samples_leaf': [1, 2, 4]
}

In [121]:
from sklearn.model_selection import StratifiedKFold
from collections import Counter
import numpy as np

def pre_validador(X, y, min_samples=5, n_splits=5, random_state=42):
    """
    Verifica se cada divisão do StratifiedKFold tem um número mínimo de amostras por classe.
    """
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    for train_index, test_index in skf.split(X, y):
        y_train = y.iloc[train_index]
        counts = Counter(y_train)
        if any(count < min_samples for count in counts.values()):
            return False  # Divisão inválida
    return True  # Todas as divisões são válidas


# Usando o pré-validador antes de executar o GridSearchCV
if pre_validador(X, y, min_samples=5, n_splits=cv.get_n_splits()):
    grid_search = GridSearchCV(pipeline, param_grid, cv=cv, scoring='f1_macro', error_score='raise')
    grid_search.fit(X, y)

    print("Melhores Hiperparâmetros:", grid_search.best_params_)
    print("Melhor F1-Score:", grid_search.best_score_)
else:
    print("Aviso: A validação cruzada não pode ser executada devido a classes com poucas amostras.")

Aviso: A validação cruzada não pode ser executada devido a classes com poucas amostras.


c:\Users\cataf\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=10.
  warnings.warn(


Ui não estamos com sorte, vamos tratar do desblancemaento das classes

### desblanceamento

In [122]:
from sklearn.utils import class_weight
import numpy as np

# Calcular os pesos das classes
class_weights = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(y),
    y=y
)
class_weight_dict = dict(zip(np.unique(y), class_weights))

model = RandomForestClassifier(random_state=42, class_weight=class_weight_dict)

In [123]:
n_splits = 5
cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Criar o modelo com pesos ajustados
model = RandomForestClassifier(random_state=42, class_weight=class_weight_dict)

# Criar um pipeline que inclui o pré-processamento e o modelo
pipeline = Pipeline([
    ('preprocessing', FunctionTransformer(preprocess_data_for_prediction, validate=False)),
    ('scaling', StandardScaler()),
    ('model', model)  # Sem reamostragem
])


In [ ]:
# Listas para armazenar as métricas de cada fold
f1_scores_macro = []
f1_scores_weighted = []
mae_scores = []
mse_scores = []
classification_reports = []

# Executar a validação cruzada e coletar as métricas
for train_index, test_index in cv.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)

    # Calcular e armazenar as métricas
    f1_macro = f1_score(y_test, y_pred, average='macro')
    f1_weighted = f1_score(y_test, y_pred, average='weighted')
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)

    f1_scores_macro.append(f1_macro)
    f1_scores_weighted.append(f1_weighted)
    mae_scores.append(mae)
    mse_scores.append(mse)

    report = classification_report(y_test, y_pred)
    classification_reports.append(report)

In [ ]:
# Imprimir as métricas médias
print("F1-Score (Macro) por fold:", f1_scores_macro)
print(f"F1-Score (Macro) Médio: {np.mean(f1_scores_macro):.4f}")
print(f"F1-Score (Macro) Desvio Padrão: {np.std(f1_scores_macro):.4f}")

print("F1-Score (Weighted) por fold:", f1_scores_weighted)
print(f"F1-Score (Weighted) Médio: {np.mean(f1_scores_weighted):.4f}")
print(f"F1-Score (Weighted) Desvio Padrão: {np.std(f1_scores_weighted):.4f}")

print("MAE por fold:", mae_scores)
print(f"MAE Médio: {np.mean(mae_scores):.4f}")
print(f"MAE Desvio Padrão: {np.std(mae_scores):.4f}")

print("MSE por fold:", mse_scores)
print(f"MSE Médio: {np.mean(mse_scores):.4f}")
print(f"MSE Desvio Padrão: {np.std(mse_scores):.4f}")

print("\nRelatório de Classificação Consolidado:")
print("------------------------------------")
for i, report in enumerate(classification_reports):
    print(f"Fold {i+1}:\n{report}")
    print("------------------------------------")

F1-Score (Macro) por fold: [0.5088340790577031, 0.5517232948079104, 0.5164619755424149, 0.40142659498126626, 0.4460969829715386]
F1-Score (Macro) Médio: 0.4849
F1-Score (Macro) Desvio Padrão: 0.0539
F1-Score (Weighted) por fold: [0.4956764546167839, 0.5263394568326879, 0.5041078723144882, 0.4466248437972733, 0.4702060975260158]
F1-Score (Weighted) Médio: 0.4886
F1-Score (Weighted) Desvio Padrão: 0.0276
MAE por fold: [1.1712962962962963, 1.337962962962963, 1.441860465116279, 1.669767441860465, 1.8325581395348838]
MAE Médio: 1.4907
MAE Desvio Padrão: 0.2352
MSE por fold: [3.810185185185185, 6.300925925925926, 8.111627906976745, 8.525581395348837, 12.111627906976745]
MSE Médio: 7.7720
MSE Desvio Padrão: 2.7341

Relatório de Classificação Consolidado:
------------------------------------
Fold 1:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       1.00      0.50      0.67         2
           2       1.00      1.00 

Com base nessas métricas, a abordagem atual (class_weight='balanced') é pior do que a linha de base (SMOTE com k_neighbors=1).

In [126]:
from sklearn.model_selection import GridSearchCV

# Definir a estratégia de validação cruzada estratificada
n_splits = 5
cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Criar o modelo
model = RandomForestClassifier(random_state=42)

# Criar um pipeline que inclui o pré-processamento, escalonamento e SMOTE
smote = SMOTE(random_state=42, k_neighbors=1)  # Manter SMOTE com k_neighbors=1
pipeline = Pipeline([
    ('preprocessing', FunctionTransformer(preprocess_data_for_prediction, validate=False)),
    ('scaling', StandardScaler()),
    ('smote', smote),
    ('model', model)
])

param_grid = {
    'model__n_estimators': [100, 200, 300],
    'model__max_depth': [5, 10, 15],
    'model__min_samples_split': [2, 5, 10],
    'model__min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(pipeline, param_grid, cv=cv, scoring='f1_macro', error_score='raise')  # Usar error_score='raise' para identificar erros
grid_search.fit(X, y)

print("Melhores Hiperparâmetros:", grid_search.best_params_)
print("Melhor F1-Score:", grid_search.best_score_)

c:\Users\cataf\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


Pré-processamento: Dimensões iniciais dos novos dados: (861, 20)
Pré-processamento: Removendo colunas irrelevantes...
  Coluna 'project_id' removida.
  Coluna 'group_id' removida.
  Coluna 'mention_handle' removida.
  Coluna 'last_minute_commits' removida.
  Coluna 'sum_lines_per_commit' removida.
Pré-processamento: Dimensões após remoção inicial: (861, 15)
Pré-processamento: Criando features _log diretas...
  Feature 'sum_lines_added_log' criada.
  Feature 'total_issues_created_log' criada.
  Feature 'issue_participation_log' criada.
Pré-processamento: Iniciando engenharia de features (ratios, combinadas)...
  Feature 'merged_ratio_log' criada.
  Feature 'issue_resolution_rate_created_log' criada.
  Feature 'total_review_activity_log' criada.
Pré-processamento: Selecionando e ordenando features finais...
Pré-processamento: Dimensões finais dos dados prontos para o modelo (antes do scaler): (861, 8)
Pré-processamento: Dimensões iniciais dos novos dados: (216, 20)
Pré-processamento: Rem

In [127]:
print("Melhores Hiperparâmetros:", grid_search.best_params_)
print("Melhor F1-Score:", grid_search.best_score_)

Melhores Hiperparâmetros: {'model__max_depth': 15, 'model__min_samples_leaf': 1, 'model__min_samples_split': 5, 'model__n_estimators': 200}
Melhor F1-Score: 0.5012648633076682


---
### vamos analisar os melhores hiperparametros com mais detalhe

In [129]:
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

# Melhores hiperparâmetros encontrados pelo GridSearchCV
melhores_parametros = grid_search.best_params_

# Criar um dicionário para armazenar os hiperparâmetros do modelo
modelo_parametros = {}

# Iterar sobre os melhores parâmetros e remover o prefixo 'model__'
for chave, valor in melhores_parametros.items():
    modelo_parametros[chave.replace('model__', '')] = valor

# Criar o modelo com os melhores hiperparâmetros
melhor_modelo = RandomForestClassifier(random_state=42, **modelo_parametros)

# Criar um pipeline que inclui o pré-processamento, escalonamento e SMOTE
smote = SMOTE(random_state=42, k_neighbors=1)  # Manter SMOTE com k_neighbors=1
melhor_pipeline = Pipeline([
    ('preprocessing', FunctionTransformer(preprocess_data_for_prediction, validate=False)),
    ('scaling', StandardScaler()),
    ('smote', smote),
    ('model', melhor_modelo)
])

In [ ]:
# Listas para armazenar as métricas de cada fold
f1_scores_macro = []
f1_scores_weighted = []
mae_scores = []
mse_scores = []
classification_reports = []

# Executar a validação cruzada e coletar as métricas
for train_index, test_index in cv.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    melhor_pipeline.fit(X_train, y_train)
    y_pred = melhor_pipeline.predict(X_test)

    # Calcular e armazenar as métricas
    f1_macro = f1_score(y_test, y_pred, average='macro')
    f1_weighted = f1_score(y_test, y_pred, average='weighted')
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)

    f1_scores_macro.append(f1_macro)
    f1_scores_weighted.append(f1_weighted)
    mae_scores.append(mae)
    mse_scores.append(mse)

    report = classification_report(y_test, y_pred)
    classification_reports.append(report)

In [131]:
# Imprimir as métricas médias
print("F1-Score (Macro) por fold:", f1_scores_macro)
print(f"F1-Score (Macro) Médio: {np.mean(f1_scores_macro):.4f}")
print(f"F1-Score (Macro) Desvio Padrão: {np.std(f1_scores_macro):.4f}")

print("F1-Score (Weighted) por fold:", f1_scores_weighted)
print(f"F1-Score (Weighted) Médio: {np.mean(f1_scores_weighted):.4f}")
print(f"F1-Score (Weighted) Desvio Padrão: {np.std(f1_scores_weighted):.4f}")

print("MAE por fold:", mae_scores)
print(f"MAE Médio: {np.mean(mae_scores):.4f}")
print(f"MAE Desvio Padrão: {np.std(mae_scores):.4f}")

print("MSE por fold:", mse_scores)
print(f"MSE Médio: {np.mean(mse_scores):.4f}")
print(f"MSE Desvio Padrão: {np.std(mse_scores):.4f}")

print("\nRelatório de Classificação Consolidado:")
print("------------------------------------")
for i, report in enumerate(classification_reports):
    print(f"Fold {i+1}:\n{report}")
    print("------------------------------------")

F1-Score (Macro) por fold: [0.49927593140732207, 0.5652257139505725, 0.530711013359483, 0.45282644665599614, 0.45828521116496745]
F1-Score (Macro) Médio: 0.5013
F1-Score (Macro) Desvio Padrão: 0.0428
F1-Score (Weighted) por fold: [0.4741330849779653, 0.5496997862906556, 0.47197043342615697, 0.4615099005098124, 0.477445991624405]
F1-Score (Weighted) Médio: 0.4870
F1-Score (Weighted) Desvio Padrão: 0.0318
MAE por fold: [1.2222222222222223, 1.3796296296296295, 1.4790697674418605, 1.6325581395348838, 1.7023255813953488]
MAE Médio: 1.4832
MAE Desvio Padrão: 0.1727
MSE por fold: [4.583333333333333, 6.842592592592593, 7.469767441860465, 7.586046511627907, 9.786046511627907]
MSE Médio: 7.2536
MSE Desvio Padrão: 1.6655

Relatório de Classificação Consolidado:
------------------------------------
Fold 1:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       0.50      0.50      0.50         2
           2       1.00      1.

Melhorou para f1 score mas piorou para MAE E MSE entao secalhar (de certeza) estava a usar no grid search a métrica errada... vamos tentar outra vez

In [132]:
from sklearn.metrics import make_scorer, mean_absolute_error

# Criar um scorer para MAE (o GridSearchCV minimiza a pontuação, então usamos o negativo do MAE)
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)

grid_search = GridSearchCV(pipeline, param_grid, cv=cv, scoring=mae_scorer, error_score='raise')
grid_search.fit(X, y)

c:\Users\cataf\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


Pré-processamento: Dimensões iniciais dos novos dados: (861, 20)
Pré-processamento: Removendo colunas irrelevantes...
  Coluna 'project_id' removida.
  Coluna 'group_id' removida.
  Coluna 'mention_handle' removida.
  Coluna 'last_minute_commits' removida.
  Coluna 'sum_lines_per_commit' removida.
Pré-processamento: Dimensões após remoção inicial: (861, 15)
Pré-processamento: Criando features _log diretas...
  Feature 'sum_lines_added_log' criada.
  Feature 'total_issues_created_log' criada.
  Feature 'issue_participation_log' criada.
Pré-processamento: Iniciando engenharia de features (ratios, combinadas)...
  Feature 'merged_ratio_log' criada.
  Feature 'issue_resolution_rate_created_log' criada.
  Feature 'total_review_activity_log' criada.
Pré-processamento: Selecionando e ordenando features finais...
Pré-processamento: Dimensões finais dos dados prontos para o modelo (antes do scaler): (861, 8)
Pré-processamento: Dimensões iniciais dos novos dados: (216, 20)
Pré-processamento: Rem

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             error_score='raise',
             estimator=Pipeline(steps=[('preprocessing',
                                        FunctionTransformer(func=<function preprocess_data_for_prediction at 0x00000198484A67A0>)),
                                       ('scaling', StandardScaler()),
                                       ('smote',
                                        SMOTE(k_neighbors=1, random_state=42)),
                                       ('model',
                                        RandomForestClassifier(random_state=42))]),
             param_grid={'model__max_depth': [5, 10, 15],
                         'model__min_samples_leaf': [1, 2, 4],
                         'model__min_samples_split': [2, 5, 10],
                         'model__n_estimators': [100, 200, 300]},
             scoring=make_scorer(mean_absolute_error, greater_is_better=False, response_method='predict'))

In [133]:
print("Melhores Hiperparâmetros:", grid_search.best_params_)
print("Melhor MAE:", -grid_search.best_score_)  # Inverter o sinal para obter o MAE positivo

Melhores Hiperparâmetros: {'model__max_depth': 15, 'model__min_samples_leaf': 1, 'model__min_samples_split': 2, 'model__n_estimators': 200}
Melhor MAE: 1.4691559000861327


top

In [134]:
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.metrics import make_scorer

# Melhores hiperparâmetros encontrados pelo GridSearchCV
melhores_parametros = grid_search.best_params_

# Criar um dicionário para armazenar os hiperparâmetros do modelo
modelo_parametros = {}

# Iterar sobre os melhores parâmetros e remover o prefixo 'model__'
for chave, valor in melhores_parametros.items():
    modelo_parametros[chave.replace('model__', '')] = valor

# Criar o modelo com os melhores hiperparâmetros
melhor_modelo = RandomForestClassifier(random_state=42, **modelo_parametros)

# Criar um pipeline que inclui o pré-processamento, escalonamento e SMOTE
smote = SMOTE(random_state=42, k_neighbors=1)  # Manter SMOTE com k_neighbors=1
melhor_pipeline = Pipeline([
    ('preprocessing', FunctionTransformer(preprocess_data_for_prediction, validate=False)),
    ('scaling', StandardScaler()),
    ('smote', smote),
    ('model', melhor_modelo)
])

In [ ]:
# Listas para armazenar as métricas de cada fold
f1_scores_macro = []
f1_scores_weighted = []
mae_scores = []
mse_scores = []
classification_reports = []

# Executar a validação cruzada e coletar as métricas
for train_index, test_index in cv.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    melhor_pipeline.fit(X_train, y_train)
    y_pred = melhor_pipeline.predict(X_test)

    # Calcular e armazenar as métricas
    f1_macro = f1_score(y_test, y_pred, average='macro')
    f1_weighted = f1_score(y_test, y_pred, average='weighted')
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)

    f1_scores_macro.append(f1_macro)
    f1_scores_weighted.append(f1_weighted)
    mae_scores.append(mae)
    mse_scores.append(mse)

    report = classification_report(y_test, y_pred)
    classification_reports.append(report)

In [136]:
# Imprimir as métricas médias
print("F1-Score (Macro) por fold:", f1_scores_macro)
print(f"F1-Score (Macro) Médio: {np.mean(f1_scores_macro):.4f}")
print(f"F1-Score (Macro) Desvio Padrão: {np.std(f1_scores_macro):.4f}")

print("F1-Score (Weighted) por fold:", f1_scores_weighted)
print(f"F1-Score (Weighted) Médio: {np.mean(f1_scores_weighted):.4f}")
print(f"F1-Score (Weighted) Desvio Padrão: {np.std(f1_scores_weighted):.4f}")

print("MAE por fold:", mae_scores)
print(f"MAE Médio: {np.mean(mae_scores):.4f}")
print(f"MAE Desvio Padrão: {np.std(mae_scores):.4f}")

print("MSE por fold:", mse_scores)
print(f"MSE Médio: {np.mean(mse_scores):.4f}")
print(f"MSE Desvio Padrão: {np.std(mse_scores):.4f}")

print("\nRelatório de Classificação Consolidado:")
print("------------------------------------")
for i, report in enumerate(classification_reports):
    print(f"Fold {i+1}:\n{report}")
    print("------------------------------------")

F1-Score (Macro) por fold: [0.49032573287357323, 0.5529740040433456, 0.5459279326991281, 0.44855326165662307, 0.45879223842552747]
F1-Score (Macro) Médio: 0.4993
F1-Score (Macro) Desvio Padrão: 0.0432
F1-Score (Weighted) por fold: [0.47270036231503676, 0.5399711774857947, 0.5076102377911227, 0.47005928373041356, 0.4904874761670525]
F1-Score (Weighted) Médio: 0.4962
F1-Score (Weighted) Desvio Padrão: 0.0257
MAE por fold: [1.3101851851851851, 1.3472222222222223, 1.3813953488372093, 1.6418604651162791, 1.6651162790697673]
MAE Médio: 1.4692
MAE Desvio Padrão: 0.1524
MSE por fold: [5.078703703703703, 6.541666666666667, 7.148837209302326, 8.162790697674419, 9.61860465116279]
MSE Médio: 7.3101
MSE Desvio Padrão: 1.5266

Relatório de Classificação Consolidado:
------------------------------------
Fold 1:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       0.50      0.50      0.50         2
           2       1.00      

mau

O melhor modelo até agora foi:
- Modelo: RandomForestClassifier com hiperparâmetros padrão.
- Reamostragem: SMOTE com k_neighbors=1.

vou guarda-lo apenas para ver a previsão no testar_modelo para evr se ele consegueria prever bem o nosso

In [138]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline  # Importe Pipeline do imblearn
from sklearn.preprocessing import FunctionTransformer
import joblib
from collections import Counter

# Lista final de features que o modelo treinado espera, NA ORDEM CORRETA.
FEATURES_FINAIS_PARA_MODELO = [
    'total_issues_created_log',
    'issue_participation_log',
    'issue_resolution_rate_created_log',
    'total_review_activity_log',
    'merged_ratio_log',
    'sum_lines_added_log',
    'interval',
    'branches_created',
]

# Epsilon para evitar divisão por zero nos ratios
EPSILON = 1e-6

# Colunas a serem removidas inicialmente se existirem
COLS_TO_DROP_INITIAL = [
    "project_id",
    "group_id",
    "mention_handle",
    "last_minute_commits",
    "sum_lines_per_commit"
]

def preprocess_data_for_prediction(df_new_raw: pd.DataFrame) -> pd.DataFrame:
    df_processed = df_new_raw.copy()
    print(f"Pré-processamento: Dimensões iniciais dos novos dados: {df_processed.shape}")

    for col in COLS_TO_DROP_INITIAL:
        if col in df_processed.columns:
            df_processed.drop(columns=[col], inplace=True)
            print(f"  Coluna '{col}' removida.")
    print(f"Pré-processamento: Dimensões após remoção inicial: {df_processed.shape}")

    if 'sum_lines_added' in df_processed.columns:
        df_processed['sum_lines_added_log'] = np.log1p(df_processed['sum_lines_added'])
        print("  Feature 'sum_lines_added_log' criada.")
    else:
        print("  AVISO: Coluna 'sum_lines_added' não encontrada. 'sum_lines_added_log' não será criada.")

    if 'total_issues_created' in df_processed.columns:
        df_processed['total_issues_created_log'] = np.log1p(df_processed['total_issues_created'])
        print("  Feature 'total_issues_created_log' criada.")
    else:
        print("  AVISO: Coluna 'total_issues_created' não encontrada. 'total_issues_created_log' não será criada.")

    if 'issue_participation' in df_processed.columns:
        df_processed['issue_participation_log'] = np.log1p(df_processed['issue_participation'])
        print("  Feature 'issue_participation_log' criada.")
    else:
        print("  AVISO: Coluna 'issue_participation' não encontrada. 'issue_participation_log' não será criada.")

    if 'merged_requests' in df_processed.columns and 'total_merge_requests' in df_processed.columns:
        merged_ratio_raw = df_processed['merged_requests'] / (df_processed['total_merge_requests'] + EPSILON)
        df_processed['merged_ratio_log'] = np.log1p(merged_ratio_raw)
        print("  Feature 'merged_ratio_log' criada.")
    else:
        print("  AVISO: Colunas 'merged_requests' ou 'total_merge_requests' não encontradas. 'merged_ratio_log' não será criada.")
        df_processed['merged_ratio_log'] = np.log1p(0)

    if 'issues_resolved' in df_processed.columns and 'total_issues_created' in df_processed.columns:
        issue_res_rate_created_raw = df_processed['issues_resolved'] / (df_processed['total_issues_created'] + EPSILON)
        df_processed['issue_resolution_rate_created_log'] = np.log1p(issue_res_rate_created_raw)
        print("  Feature 'issue_resolution_rate_created_log' criada.")
    else:
        print("  AVISO: Colunas 'issues_resolved' ou 'total_issues_created' não encontradas. 'issue_resolution_rate_created_log' não será criada.")
        df_processed['issue_resolution_rate_created_log'] = np.log1p(0)

    if 'review_comments_given' in df_processed.columns and 'review_comments_received' in df_processed.columns:
        total_review_raw = df_processed['review_comments_given'] + df_processed['review_comments_received']
        df_processed['total_review_activity_log'] = np.log1p(total_review_raw)
        print("  Feature 'total_review_activity_log' criada.")
    else:
        print("  AVISO: Colunas 'review_comments_given' ou 'review_comments_received' não encontradas. 'total_review_activity_log' não será criada.")
        df_processed['total_review_activity_log'] = np.log1p(0)

    print("Pré-processamento: Selecionando e ordenando features finais...")
    missing_features_in_final_step = []
    for final_feature in FEATURES_FINAIS_PARA_MODELO:
        if final_feature not in df_processed.columns:
            print(f"  AVISO FINAL: Feature final '{final_feature}' não encontrada nos dados processados. Preenchendo com 0.")
            df_processed[final_feature] = 0
            missing_features_in_final_step.append(final_feature)

    if missing_features_in_final_step:
        print(f"  Resumo de features finais que foram preenchidas com 0: {missing_features_in_final_step}")

    for col in FEATURES_FINAIS_PARA_MODELO:
        if col not in df_processed:
            df_processed[col] = 0

    df_model_ready = df_processed[FEATURES_FINAIS_PARA_MODELO]
    print(f"Pré-processamento: Dimensões finais dos dados prontos para o modelo (antes do scaler): {df_model_ready.shape}")

    return df_model_ready

# Carregar os dados
df = pd.read_csv("student_features_interval_all.csv")

# Separar features e target
X = df.drop(columns=["Final Grade"])
y = df["Final Grade"]

# Criar o modelo
model = RandomForestClassifier(random_state=42)  # Hiperparâmetros padrão

# Criar um pipeline que inclui o pré-processamento, escalonamento e SMOTE
smote = SMOTE(random_state=42, k_neighbors=1)
pipeline = Pipeline([
    ('preprocessing', FunctionTransformer(preprocess_data_for_prediction, validate=False)),
    ('scaling', StandardScaler()),
    ('smote', smote),
    ('model', model)
])

# Treinar o modelo em todos os dados
pipeline.fit(X, y)

# Salvar o modelo
joblib.dump(pipeline, 'melhor_modelo.pkl')

print("Modelo treinado e salvo como 'melhor_modelo.pkl'")

Pré-processamento: Dimensões iniciais dos novos dados: (1077, 20)
  Coluna 'project_id' removida.
  Coluna 'group_id' removida.
  Coluna 'mention_handle' removida.
  Coluna 'last_minute_commits' removida.
  Coluna 'sum_lines_per_commit' removida.
Pré-processamento: Dimensões após remoção inicial: (1077, 15)
  Feature 'sum_lines_added_log' criada.
  Feature 'total_issues_created_log' criada.
  Feature 'issue_participation_log' criada.
  Feature 'merged_ratio_log' criada.
  Feature 'issue_resolution_rate_created_log' criada.
  Feature 'total_review_activity_log' criada.
Pré-processamento: Selecionando e ordenando features finais...
Pré-processamento: Dimensões finais dos dados prontos para o modelo (antes do scaler): (1077, 8)
Modelo treinado e salvo como 'melhor_modelo.pkl'


o modelo prevê sempre 11 a toda a gente, independentemente do intervalo --> PESSIMO

# --

In [139]:
from sklearn.utils import resample

# Separar a classe majoritária
df_major = df_treino[df_treino["Final Grade"] == 15]

# Classes minoritárias
df_minor = df_treino[df_treino["Final Grade"] != 15]

# Undersample a maioria
df_major_downsampled = resample(df_major, replace=False, n_samples=100, random_state=42)

# Juntar
df_balanceado = pd.concat([df_major_downsampled, df_minor])


NameError: name 'df_treino' is not defined